In [246]:
import sys
import json
import pydeck as pdk
import haversine as hs

local = sys.modules[__name__]

In [ ]:
import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key=api_key)

In [247]:
wheels = {
    'W18': {
        'manufacturer': 'Shanghai Wheel Company Ltd',
        'price': 0,
        'address': '4RV5+P8J, Yingbin Expy, Pudong, Shanghai, China', # Shanghai Airport
        'weight': 90, # kg
        'materials': {
            'rubber': 10,
            'metal': 36
        }
    },
    'W19': {
        'manufacturer': 'Berlin Wheel Company Ltd',
        'price': 1000,
        'address': 'Melli-Beese-Ring 1, 12529 Schönefeld, Germany', # Berlin Airport
        'weight': 100, # kg
        'materials' : {
            'rubber': 11,
            'metal': 38,
        }
    },
    'W20': {
        'manufacturer': 'Austin Wheel Company Ltd',
        'price': 2000,
        'address': '3600 Presidential Blvd, Austin, TX 78719, USA', # Austin Airport
        'weight': 120, # kg
        'materials' : {
            'rubber': 12,
            'metal': 40
        }
    }
}

In [248]:
seats = {
    'Black Seat': {
        'manufacturer': 'Tokyo Seat Company',
        'price': 0,
        'address': '1-1 Furugome, Narita, Chiba 282-0004, Japan', # Tokyo Airport
        'weight': 100, # kg
        'materials': {
            'leather': 15,
            'metal': 20
        }
    },
    'White Seat': {
        'manufacturer': 'Mexico City Seat Company',
        'price': 1000,
        'address': 'Av. Capitan Carlos Leon S/N, Penon de los Banos, Venustiano Carranza, 15620 Ciudad de México, CDMX, Mexico', # Mexico City Airport
        'weight': 100, # kg
        'materials' : {
            'leatherette': 15,
            'metal': 20
        }
    }
}

In [249]:
models = {
    'Model H':{
        'price': 40000,
        'manufacturer': 'Hedera Motors Dallas',
        'address': '2400 Aviation Dr, DFW Airport, TX 75261, USA',
        'weight': 1800,
        'materials':{
            'metal': 1000,
            'plastic': 400,
        },
        'options': {
            'wheels': None,
            'seats': None
        }
    },
    'Model B':{
        'price': 70000,
        'manufacturer': 'Hedera Motors Hangzhou',
        'address': '6CPQ+9HC, Xiaoshan District, Hangzhou, Zhejiang, China',
        'weight': 1800,
        'materials':{
            'metal': 1100,
            'plastic': 500,
        },
        'options': {
            'wheels': None,
            'seats': None
        }
    },
    'Model A':{
        'price': 50000,
        'manufacturer': 'Hedera Motors Dallas',
        'address': '2400 Aviation Dr, DFW Airport, TX 75261, USA',
        'weight': 2100,
        'materials':{
            'metal': 1300,
            'plastic': 700,
        },
        'options': {
            'wheels': None,
            'seats': None
        }    },
    'Model R':{
        'price': 80000,
        'manufacturer': 'Hedera Motors Hangzhou',
        'address': '6CPQ+9HC, Xiaoshan District, Hangzhou, Zhejiang, China',
        'weight': 2100,
        'materials':{
            'metal': 1500,
            'plastic': 800,
        },
        'options': {
            'wheels': None,
            'seats': None
        }
    }
}

In [250]:
class Car:

    def __init__(self):
        self.car = None
        self.kgm = 0
        self.mat = {}
        self.nodes = []
        self.edges = []

    def select_model(self, model):
        self.car = json.loads(json.dumps(
            getattr(local, 'models')[model]
        ))

    def select_option(self, option, choice):
        self.car['options'][option] = json.loads(json.dumps(
            getattr(local, option)[choice]
        ))

    def get_coords(self, address):

        response = gmaps.geocode(address)

        return response[0]['geometry']['location']['lat'], response[0]['geometry']['location']['lng']  

    def set_coords(self, parent=None):
        
        if parent is None:
            parent = self.car
        
        parent['lat'], parent['lng'] = self.get_coords(parent['address'])
        
        if 'options' in parent:
            for option, child in parent['options'].items():
                self.set_coords(child)
                
    def sum_mat(self, parent=None):
        
        if parent is None:
            parent = self.car
            
        if 'materials' in parent:
            for material, amt in parent['materials'].items():
                if material not in self.mat:
                    self.mat[material] = 0
                self.mat[material] += amt
            
        if 'options' in parent:
            for option, child in parent['options'].items():
                self.sum_mat(child)
                
    def get_nodes(self, parent=None):
        
        if parent is None:
            parent = self.car
        
        self.nodes.append({k: parent[k] for k in set(list(parent.keys())) - set(['options'])})

        if 'options' in parent:
            for option, child in parent['options'].items():
                self.get_nodes(child)
                
    def calc_edges(self, parent=None):
        
        if parent is None:
            parent = self.car
            
        if 'options' in parent:
            for option, child in parent['options'].items():

                parent_coords = [parent['lng'], parent['lat']]
                child_coords = [child['lng'], child['lat']]

                self.edges.append({
                    'start': parent_coords,
                    'end': child_coords,
                    'distance': hs.haversine(
                        parent_coords[::-1],
                        child_coords[::-1])
                })
            
            self.calc_edges(child)
            
    def plot(self):
        
        layer = pdk.Layer(
            "ScatterplotLayer",
            data=car.nodes,
            get_position="[lng, lat]",
            get_color=[255, 0, 0],  # Set color to red
            radius_scale=1,
            radius_min_pixels=2,
            radius_max_pixels=10,
            get_radius=100000,  # Set radius to 100 pixels
            pickable=True,
        )

        line_layer = pdk.Layer(
            "LineLayer",
            car.edges,
            get_source_position="start",
            get_target_position="end",
            get_color=[255, 255, 255],
            get_width=3,
            highlight_color=[255, 255, 255],
            picking_radius=5,
            auto_highlight=True,
            pickable=True,
        )

        view = pdk.ViewState(latitude=0, longitude=0, min_zoom=0.5, zoom=1, max_zoom=3)

        deck = pdk.Deck(layers=[line_layer, layer], initial_view_state=view)
        
        return deck

In [251]:
car = Car()

In [252]:
car.select_model('Model H') # to be a dropdown

In [253]:
car.select_option('seats', 'White Seat') # to be a dropdown

In [254]:
car.select_option('wheels', 'W19') # to be a dropdown

In [255]:
car.set_coords() # set coords of all addresses

In [256]:
car.sum_mat() # sum materials

In [257]:
car.get_nodes() # hi

In [258]:
car.calc_edges() # artificial intelligence blockchain big data quantum computing

In [259]:
car.plot()

{
  "initialViewState": {
    "latitude": 0,
    "longitude": 0,
    "maxZoom": 3,
    "minZoom": 0.5,
    "zoom": 1
  },
  "layers": [
    {
      "@@type": "LineLayer",
      "autoHighlight": true,
      "data": [
        {
          "distance": 8396.641135536738,
          "end": [
            13.510658,
            52.364628
          ],
          "start": [
            -97.0403352,
            32.8998091
          ]
        },
        {
          "distance": 1510.6163173753025,
          "end": [
            -99.0805443,
            19.4375028
          ],
          "start": [
            -97.0403352,
            32.8998091
          ]
        }
      ],
      "getColor": [
        255,
        255,
        255
      ],
      "getSourcePosition": "@@=start",
      "getTargetPosition": "@@=end",
      "getWidth": 3,
      "highlightColor": [
        255,
        255,
        255
      ],
      "id": "b38295ac-59ea-4a5a-ad8a-fc4ab6278eae",
      "pickable": true,
      "pickingRadius": 5
    },
    {
      "@@type": "ScatterplotLayer",
      "data": [
        {
          "address": "2400 Aviation Dr, DFW Airport, TX 75261, USA",
          "lat": 32.8998091,
          "lng": -97.0403352,
          "manufacturer": "Hedera Motors Dallas",
          "materials": {
            "metal": 1000,
            "plastic": 400
          },
          "price": 40000,
          "weight": 1800
        },
        {
          "address": "Melli-Beese-Ring 1, 12529 Sch\u00f6nefeld, Germany",
          "lat": 52.364628,
          "lng": 13.510658,
          "manufacturer": "Berlin Wheel Company Ltd",
          "materials": {
            "metal": 38,
            "rubber": 11
          },
          "price": 1000,
          "weight": 100
        },
        {
          "address": "Av. Capitan Carlos Leon S/N, Penon de los Banos, Venustiano Carranza, 15620 Ciudad de M\u00e9xico, CDMX, Mexico",
          "lat": 19.4375028,
          "lng": -99.0805443,
          "manufacturer": "Mexico City Seat Company",
          "materials": {
            "leatherette": 15,
            "metal": 20
          },
          "price": 1000,
          "weight": 100
        }
      ],
      "getColor": [
        255,
        0,
        0
      ],
      "getPosition": "@@=[lng, lat]",
      "getRadius": 100000,
      "id": "ae4fa541-f602-42ef-b185-bdc0c758ca97",
      "pickable": true,
      "radiusMaxPixels": 10,
      "radiusMinPixels": 2,
      "radiusScale": 1
    }
  ],
  "mapProvider": "carto",
  "mapStyle": "https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json",
  "views": [
    {
      "@@type": "MapView",
      "controller": true
    }
  ]
}